In [2]:
import pandas as pd
import os
import requests
import numpy as np
import operator
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

In [4]:
!pwd

/Users/mos/Dropbox/memeticscience/typealyzer-dataset/notebooks/jungian_classification


In [15]:
df_pickle_path = "../../pickles/dataframe_survey_2018-01-23_enriched.pickle"

In [16]:
indata = pd.read_pickle(df_pickle_path)
indata[["actual","actual_temp","is_s"]].head(25)

,actual,actual_temp,is_s
1,INFJ,nf,0
2,INFP,nf,0
3,INTP,nt,0
5,ENFJ,nf,0
10,INFP,nf,0
11,I don't know,NaN,0
14,INFJ,nf,0
15,ENTP,nt,0
16,INTP,nt,1
17,INTP,nt,0


In [17]:
len(indata[pd.isnull(indata["tokens"])])

0

In [18]:
indata[pd.isnull(indata["tokens"])]

,url,typealyzer,actual,e,s,t,sntf_s,sntf_n,sntf_t,sntf_f,...,sad,you,cogmech,auxverb,they,incl,money,feel,we,hear


# Inspect original data function distributions

In [19]:
s_series = indata[indata.is_s == 1]["tokens"]
n_series = indata[indata.is_n == 1]["tokens"]
t_series = indata[indata.is_t == 1]["tokens"]
f_series = indata[indata.is_f == 1]["tokens"]

avg_tkns = {
    "s":s_series.mean(),
    "n":n_series.mean(),
    "t":t_series.mean(),
    "f":f_series.mean()
}
avg_tkns

{'f': 517.183945401365,
 'n': 488.49222525740703,
 's': 478.4377794336811,
 't': 519.2480824019285}

In [20]:
indata.func.value_counts()

n    9518
f    6154
t    4563
s    2684
Name: func, dtype: int64

# Ensure wordpress, blogspot and tumblr texts only

In [22]:
indata.domain.value_counts()

tumblr       21938
blogspot       513
wordpress      468
Name: domain, dtype: int64

In [39]:
indata = indata[(indata.domain == "wordpress") | (indata.domain == "blogspot") | (indata.domain == "tumblr")]
indata.func.value_counts()

n    10064
f     6545
t     4895
s     2827
Name: func, dtype: int64

# Ensure no "I don't know answers"

In [25]:
indata.actual.value_counts()

INFP            3913
INTJ            3767
INFJ            3438
INTP            3156
ISFP            1325
ENFP            1310
ISFJ            1114
ISTJ             994
ISTP             932
ENTP             734
I don't know     507
ENFJ             489
ESFP             383
ENTJ             317
ESFJ             251
ESTP             174
ESTJ             115
Name: actual, dtype: int64

In [26]:
indata = indata[indata.actual != "I don't know"]

In [27]:
indata.actual.value_counts()

INFP    3913
INTJ    3767
INFJ    3438
INTP    3156
ISFP    1325
ENFP    1310
ISFJ    1114
ISTJ     994
ISTP     932
ENTP     734
ENFJ     489
ESFP     383
ENTJ     317
ESFJ     251
ESTP     174
ESTJ     115
Name: actual, dtype: int64

# Sample equal size text chunks for training and evaluation data
Since the smallest subset ("s") contains 3073 texts (see above), I cut each class into 3000 observations.
See: [Pandas sample()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html)

In [32]:
indata.func.value_counts()

n    9311
f    6016
t    4456
s    2629
Name: func, dtype: int64

In [35]:
indata["typealyzer"] = indata.typealyzer.str.lower()
indata["actual"] = indata.actual.str.lower()
indata

,url,typealyzer,actual,e,s,t,sntf_s,sntf_n,sntf_t,sntf_f,...,sad,you,cogmech,auxverb,they,incl,money,feel,we,hear
1,http://adropofcolour.tumblr.com,isfp,infj,0.291281,0.787844,0.460961,6.635150e-01,0.178565,0.069282,8.863820e-02,...,0.000000,0.019704,0.098522,0.147783,0.000000,0.039409,0.009852,0.019704,0.044335,0.009852
2,http://godheadcomplex.tumblr.com,esfp,infp,0.883579,0.951693,0.238407,8.559210e-01,0.046931,0.021850,7.529700e-02,...,0.000000,0.017513,0.201401,0.084063,0.001751,0.056042,0.007005,0.017513,0.047285,0.003503
3,http://chaotikaeon2.tumblr.com,intj,intp,0.332444,0.357863,0.591322,1.476680e-01,0.252326,0.339831,2.601750e-01,...,0.003283,0.014540,0.181989,0.114916,0.000938,0.071295,0.010319,0.008912,0.054409,0.014540
5,http://perpetually-in-transit.blogspot.com,esfp,enfj,0.944394,0.943192,0.105527,7.788250e-01,0.051134,0.017299,1.527420e-01,...,0.002497,0.018727,0.207241,0.104869,0.002497,0.049938,0.014981,0.011236,0.041199,0.017478
10,http://museofmystery.wordpress.com/2012/08/29/...,istp,infp,0.073352,0.850472,0.608812,6.283220e-01,0.112762,0.149270,1.096460e-01,...,0.001031,0.005155,0.215464,0.122680,0.005155,0.043299,0.019588,0.002062,0.021649,0.012371
14,http://greeneyederotic.tumblr.com,istp,infj,0.387014,0.849547,0.791709,5.189260e-01,0.095035,0.299708,8.633110e-02,...,0.000000,0.073529,0.250000,0.161765,0.000000,0.014706,0.058824,0.029412,0.058824,0.000000
15,http://random-nerdy-stuff.tumblr.com,estp,entp,0.967408,0.985934,0.987147,8.820920e-01,0.011472,0.104962,1.473170e-03,...,0.007832,0.018720,0.187393,0.119580,0.003056,0.056160,0.012607,0.012607,0.031137,0.011270
16,http://supernoondles.tumblr.com,isfp,intp,0.183238,0.732849,0.267514,5.222770e-01,0.192229,0.071700,2.137940e-01,...,0.004301,0.050538,0.229032,0.148387,0.003226,0.060215,0.009677,0.019355,0.045161,0.011828
17,http://supernoondles.tumblr.com/tagged/writing,esfp,intp,0.730095,0.913702,0.076379,7.110900e-01,0.067741,0.016356,2.048130e-01,...,0.012602,0.015838,0.225817,0.148672,0.008345,0.067268,0.012091,0.012602,0.035422,0.017371
18,http://kanieu.tumblr.com,intp,intj,0.009344,0.034724,0.991369,2.802000e-02,0.688265,0.280998,2.717330e-03,...,0.005979,0.018565,0.222782,0.140340,0.005664,0.069855,0.006293,0.013845,0.025802,0.021082


In [36]:
# order: s,n,t,f
samples = pd.concat([
            indata[indata.is_s == 1].sample(2600, random_state=123456)[["text","tokens","typealyzer","actual","func","funcatt","actual_temp"]],
            indata[indata.is_n == 1].sample(2600, random_state=123456)[["text","tokens","typealyzer","actual","func","funcatt","actual_temp"]],
            indata[indata.is_t == 1].sample(2600, random_state=123456)[["text","tokens","typealyzer","actual","func","funcatt","actual_temp"]],
            indata[indata.is_f == 1].sample(2600, random_state=123456)[["text","tokens","typealyzer","actual","func","funcatt","actual_temp"]]
            ])

In [37]:
samples.head(3)

,text,tokens,typealyzer,actual,func,funcatt,actual_temp
19885,ja kaikki vois olla ihanasti nyt Follow on Tum...,662,istp,infj,s,se,nf
13252,Minim♥listic Minim♥listic Archive Random RSS S...,272,istp,infp,s,si,nf
8263,trusting in allegories >>>>>>>>>>>>>>>>>>>>>>>...,751,esfp,istj,s,si,st


In [38]:
samples.func.value_counts()

t    2600
s    2600
f    2600
n    2600
Name: func, dtype: int64

In [39]:
samples.to_pickle("jung_functions_samples_blogs_n2600.pickle")

In [3]:
samples = pd.read_pickle("jung_functions_samples_blogs_n2600.pickle")

# Setup uClassify classifer and prepare training and evaluation datasets
The variable `os.environ["UCLASSIFY_WRITE"]` is created by adding a line to e.g. `~/.profile`:

`export UCLASSIFY_WRITE = "<your_uclassify_write_key>"`

In [148]:
def setup_jung_functions_en_classes():
    """The classifier is already created manually at https://uclassify.com/manage/classifiers/Jung-Cognitive-Functions-En/.
    """
    for name in ["s","n","t","f"]:
        data = {"key":os.environ["UCLASSIFY_WRITE"],"className":name}
        header = {"Content-Type": "application/json",
                  "Authorization": "Token " + os.environ["UCLASSIFY_WRITE"]}

        response = requests.post('https://api.uclassify.com/v1/me/Jung-Cognitive-Functions-En/addClass', 
                        json = data,
                        headers = header)
    return print("name: {}, response: {}".format(name, response.content))

setup_jung_functions_en_classes()

name: f, response: b''


# Split SNTF samples into train and eval subsets.

In [40]:
# prepare column to keep track of what's been used for training
zeros = np.zeros(len(samples))
samples["training_set"] = zeros
samples["training_set"] = samples.training_set.astype("int")
samples.head(3)

,text,tokens,typealyzer,actual,func,funcatt,actual_temp,training_set
19885,ja kaikki vois olla ihanasti nyt Follow on Tum...,662,istp,infj,s,se,nf,0
13252,Minim♥listic Minim♥listic Archive Random RSS S...,272,istp,infp,s,si,nf,0
8263,trusting in allegories >>>>>>>>>>>>>>>>>>>>>>>...,751,esfp,istj,s,si,st,0


In [41]:
traing_set_size = 1800 # e.g. 1800 is about 70% of 2600 samples
s_train = samples[samples.func == "s"].sample(traing_set_size).index
n_train = samples[samples.func == "n"].sample(traing_set_size).index
t_train = samples[samples.func == "t"].sample(traing_set_size).index
f_train = samples[samples.func == "f"].sample(traing_set_size).index

sn_train = s_train.union(n_train)
tf_train = t_train.union(f_train)
sntf_train = sn_train.union(tf_train)

samples.loc[sntf_train, "training_set"] = 1
samples.head(15)

,text,tokens,typealyzer,actual,func,funcatt,actual_temp,training_set
19885,ja kaikki vois olla ihanasti nyt Follow on Tum...,662,istp,infj,s,se,nf,1
13252,Minim♥listic Minim♥listic Archive Random RSS S...,272,istp,infp,s,si,nf,0
8263,trusting in allegories >>>>>>>>>>>>>>>>>>>>>>>...,751,esfp,istj,s,si,st,1
2162,Peek into what's in my brain. Peek into what's...,158,isfp,infp,s,si,nf,1
14796,"fortune favors the brave, dude home message ab...",347,intp,infj,s,si,nf,0
5861,Sit back and watch the awkward<semic> here the...,469,isfp,intj,s,si,nt,1
12770,Im In A Björk Phase Right Now aristocratslayer...,412,esfp,isfp,s,si,sf,0
23579,"greedy for power, for honor, for love. -------...",325,istp,esfj,s,si,sf,0
24741,purity purity “Imagine a man standing at a clo...,1178,intp,intj,s,se,nt,0
2602,on va voir on va voir hi i'm caro and my favou...,85,esfp,infj,s,si,nf,1


In [42]:
len(samples)

10400

In [43]:
samples.func.value_counts()

t    2600
s    2600
f    2600
n    2600
Name: func, dtype: int64

In [44]:
samples[samples.training_set == 1]["func"].value_counts()

n    1800
t    1800
s    1800
f    1800
Name: func, dtype: int64

# Split percieving and judging samples into train and eval subsets.

In [55]:
traing_set_size = 2000 # e.g. 2000 is about 70% of 2800 samples
s_train = samples[samples.func == "s"].sample(traing_set_size).index
n_train = samples[samples.func == "n"].sample(traing_set_size).index
t_train = samples[samples.func == "t"].sample(traing_set_size).index
f_train = samples[samples.func == "f"].sample(traing_set_size).index

sn_train = s_train.union(n_train)
tf_train = t_train.union(f_train)
sntf_train = sn_train.union(tf_train)

samples.loc[sntf_train, "training_set"] = 1
samples.head(15)

,text,tokens,typealyzer,actual,func,funcatt,actual_temp,training_set
19885,ja kaikki vois olla ihanasti nyt Follow on Tum...,662,istp,infj,s,se,nf,1
13252,Minim♥listic Minim♥listic Archive Random RSS S...,272,istp,infp,s,si,nf,0
8263,trusting in allegories >>>>>>>>>>>>>>>>>>>>>>>...,751,esfp,istj,s,si,st,1
2162,Peek into what's in my brain. Peek into what's...,158,isfp,infp,s,si,nf,1
14796,"fortune favors the brave, dude home message ab...",347,intp,infj,s,si,nf,1
5861,Sit back and watch the awkward<semic> here the...,469,isfp,intj,s,si,nt,1
12770,Im In A Björk Phase Right Now aristocratslayer...,412,esfp,isfp,s,si,sf,1
23579,"greedy for power, for honor, for love. -------...",325,istp,esfj,s,si,sf,0
24741,purity purity “Imagine a man standing at a clo...,1178,intp,intj,s,se,nt,1
2602,on va voir on va voir hi i'm caro and my favou...,85,esfp,infj,s,si,nf,1


# Train Four-dimension (SNTF) and two-dimensions (SN and TF) uClassify.com classifiers

In [68]:
def train_jung_cognitive_functions_en_classes(func, classifier):
    """Presupposes that classifier is created and that setup_jung_functions_en_classes() is already run.
    func: expects one of ["s","n","t","f"]
    classifier: expects on of ["sntf", "tf", "sn"]
    
    """
    trained_ix = []
    text_count = 1
    for ix, row in samples.loc[(samples.func == name) & (samples.training_set == 1)].iterrows():
        trained_ix.append(ix)
        data = {"texts":[row["text"]]}
        header = {"Content-Type": "application/json",
                 "Authorization": "Token " + os.environ["UCLASSIFY_WRITE"]}
        if classifier == "sntf":
            response = requests.post('https://api.uclassify.com/v1/me/Jung-Cognitive-Functions-En/' + name + "/train", 
                    json = data,
                    headers = header)
            if text_count % 100 == 0:
                print("{}:{}".format(name, text_count))
        
        elif classifier == "sn":
            response = requests.post('https://api.uclassify.com/v1/me/jung-percieving-2000/' + name + "/train", 
                    json = data,
                    headers = header)
            if text_count % 100 == 0:
                print("{}:{}".format(name, text_count))
        
        elif classifier == "tf":
            response = requests.post('https://api.uclassify.com/v1/me/jung-judging-2000/' + name + "/train", 
                    json = data,
                    headers = header)
            if text_count % 100 == 0:
                print("{}:{}".format(name, text_count))
        
        text_count += 1
    
    print("Finished training Jung Cognitive Function: {}".format(name))
    return trained_ix

## Train SNTF classifier

## Train SN classifer

## Train TF classifier

In [69]:
tf_trained_ix = []
for name in ["t","f"]:
    functions_trained_ix = train_jung_cognitive_functions_en_classes(name, classifier="tf")
    tf_trained_ix.append(functions_trained_ix)

t:100
t:200
t:300
t:400
t:500
t:600
t:700
t:800
t:900
t:1000
t:1100
t:1200
t:1300
t:1400
t:1500
t:1600
t:1700
t:1800
t:1900
t:2000
t:2100
t:2200
t:2300
t:2400
Finished training Jung Cognitive Function: t
f:100
f:200
f:300
f:400
f:500
f:600
f:700
f:800
f:900
f:1000
f:1100
f:1200
f:1300
f:1400
f:1500
f:1600
f:1700
f:1800
f:1900
f:2000
f:2100
f:2200
f:2300
f:2400
Finished training Jung Cognitive Function: f


In [61]:
# prepare column to keep track of what's been used for training
eval_set = samples[samples.training_set == 0]
eval_set.head(3)

,text,tokens,typealyzer,actual,func,funcatt,actual_temp,training_set
13252,Minim♥listic Minim♥listic Archive Random RSS S...,272,istp,infp,s,si,nf,0
23579,"greedy for power, for honor, for love. -------...",325,istp,esfj,s,si,sf,0
4251,thrash unreal get off my lawn you damn kids //...,118,esfp,infp,s,si,nf,0


In [62]:
len(samples)

10400

0

In [63]:
len(eval_set)

711

# Classify SNTF

In [65]:
def classify_jung_cognitive_function_of_text(text):
    """Does what it says, pretty much."""
    header = {"Content-Type": "application/json",
             "Authorization": "Token " + os.environ["UCLASSIFY_READ"]}
    data = {"texts":[text]} # send a one-item list for now, since we don't have a feel for sizes
    result = requests.post("https://api.uclassify.com/v1/prfekt/jung-cognitive-functions-en/classify",
                       json = data,
                       headers = header)
    json_result = result.json()
    
    res_dict = {"s":0, "n":0, "t":0, "f":0}
    
    for classItem in json_result[0]["classification"]:
        res_dict[classItem["className"]] = classItem["p"]
    
    sorted_dict = sorted(res_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_dict

In [66]:
zeros = np.zeros(len(eval_set))
sntf_results = []
sntf2_results = []
row_cnt = 1
for ix, row in eval_set.iterrows():
    print("row: {} of {}".format(row_cnt, len(eval_set)),end="\r")
    res = classify_jung_cognitive_function_of_text(row["text"])
    sntf_results.append(res[0][0])
    sntf2_results.append(res[0][1])
    row_cnt += 1

KeyboardInterrupt: 

# Classify SN 

In [56]:
def classify_jung_percieving_function_of_text(text):
    """Does what it says, pretty much."""
    header = {"Content-Type": "application/json",
             "Authorization": "Token " + os.environ["UCLASSIFY_READ"]}
    data = {"texts":[text]} # send a one-item list for now, since we don't have a feel for sizes
    result = requests.post("https://api.uclassify.com/v1/prfekt/jung-percieving-2000/classify",
                       json = data,
                       headers = header)
    json_result = result.json()
    
    res_dict = {"s":0, "n":0}
    
    for classItem in json_result[0]["classification"]:
        res_dict[classItem["className"]] = classItem["p"]
    
    sorted_dict = sorted(res_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_dict

In [57]:
zeros = np.zeros(len(eval_set))
sn_results = []
row_cnt = 1
for ix, row in eval_set.iterrows():
    print("row: {} of {}".format(row_cnt, len(eval_set)),end="\r")
    res = classify_jung_percieving_function_of_text(row["text"])
    sn_results.append(res[0][0])
    row_cnt += 1

# Classify TF

In [ ]:
def classify_jung_judging_function_of_text(text):
    """Does what it says, pretty much."""
    header = {"Content-Type": "application/json",
             "Authorization": "Token " + os.environ["UCLASSIFY_READ"]}
    data = {"texts":[text]} # send a one-item list for now, since we don't have a feel for sizes
    result = requests.post("https://api.uclassify.com/v1/prfekt/jung-judging-2000/classify",
                       json = data,
                       headers = header)
    json_result = result.json()
    
    res_dict = {"t":0, "f":0}
    
    for classItem in json_result[0]["classification"]:
        res_dict[classItem["className"]] = classItem["p"]
    
    sorted_dict = sorted(res_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_dict

In [59]:
zeros = np.zeros(len(eval_set))
tf_results = []
row_cnt = 1
for ix, row in eval_set.iterrows():
    print("row: {} of {}".format(row_cnt, len(eval_set)),end="\r")
    res = classify_jung_judging_function_of_text(row["text"])
    tf_results.append(res[0][0])
    row_cnt += 1

# Evaluation

In [79]:
# If tampered with
eval_set = pd.read_pickle("classification_results_blogs_n2800_dataframe.pickle")

In [53]:
eval_set.head(3)

,text,tokens,typealyzer,actual,func,funcatt,actual_temp,training_set
13252,Minim♥listic Minim♥listic Archive Random RSS S...,272,istp,infp,s,si,nf,0
14796,"fortune favors the brave, dude home message ab...",347,intp,infj,s,si,nf,0
12770,Im In A Björk Phase Right Now aristocratslayer...,412,esfp,isfp,s,si,sf,0


In [54]:
eval_set = pd.concat([eval_set,
                      pd.DataFrame(sntf_results, index=eval_set.index),
                      pd.DataFrame(sn_results, index=eval_set.index),
                      pd.DataFrame(tf_results, index=eval_set.index)
                     ], axis=1, ignore_index=True)
eval_set.columns = ["text","tokens","func","funcatt","actual_temp","training_set","sntf","sn","tf"]
eval_set.to_pickle("classification_results_blogs_n2800_dataframe.pickle")
eval_set.head(100)

NameError: name 'sn_results' is not defined

# Classification report SNTF

In [62]:
sntf_cr = classification_report(eval_set['func'], eval_set['sntf'])
print(sntf_cr)

             precision    recall  f1-score   support

          f       0.57      0.56      0.56       800
          n       0.65      0.42      0.51       800
          s       0.57      0.82      0.67       800
          t       0.63      0.57      0.60       800

avg / total       0.60      0.59      0.59      3200



In [78]:
sntf_accuracy = sum(eval_set['func']==eval_set['sntf'])/len(df)
print(sntf_accuracy)

0.5934375


# Add columns percieving and judging for evaluation of s/n and t/f classifiers

In [89]:
eval_set["perc_func"] = eval_set.actual_temp.str.extract("(\w)\w", expand=False)
eval_set["judg_func"] = eval_set.actual_temp.str.extract("\w(\w)", expand=False)

# Classification report temperament (s/n vs t/f classifiers)

In [82]:
eval_set["classified_temp"] = eval_set["sn"] + eval_set["tf"]
temp_cr = classification_report(eval_set['actual_temp'], eval_set['classified_temp'])
print(temp_cr)

             precision    recall  f1-score   support

         nf       0.51      0.32      0.39      1004
         nt       0.53      0.39      0.45      1014
         sf       0.30      0.55      0.39       646
         st       0.26      0.31      0.28       536

avg / total       0.43      0.39      0.39      3200



In [83]:
temp_accuracy = sum(eval_set['actual_temp']==eval_set['classified_temp'])/len(df)
print(temp_accuracy)

0.3859375


In [84]:
eval_set.classified_temp.value_counts()

sf    1190
nt     752
st     629
nf     629
Name: classified_temp, dtype: int64

# Classification report temperament (typealyzer classifier)

In [ ]:
eval_set["typealyzer_temp"] = eval_set[]

# Classification report SN

In [90]:
perc_cr = classification_report(eval_set["perc_func"], eval_set["sn"])
print(perc_cr)

             precision    recall  f1-score   support

          n       0.82      0.56      0.66      2018
          s       0.51      0.79      0.62      1182

avg / total       0.71      0.64      0.65      3200



# Classification report TF

In [91]:
judg_cr = classification_report(eval_set["judg_func"], eval_set["tf"])
print(judg_cr)

             precision    recall  f1-score   support

          f       0.62      0.68      0.65      1650
          t       0.62      0.55      0.58      1550

avg / total       0.62      0.62      0.61      3200



In [ ]:
classes=list(set(eval_set['uclassify'].values))
cm=pd.DataFrame(confusion_matrix(eval_set['func'].values,eval_set['uclassify'].values,labels=classes))
cm.columns=classes
cm.index=classes
cm

In [ ]:
for ix, row in eval_set.iterrows():
    res = classify_jung_cognitive_function_of_text(row["text"])
    eval_set.loc[ix, "uclassify"] = res[]

In [ ]:
eval_set = pd.concat([eval_set,pd.Series(results)], axis=1, ignore_index=True)

In [ ]:
len(results)

In [ ]:
eval_set.groupby(['uclassify','func']).size()

In [ ]:
actual = eval_set['func'].value_counts()
predicted = eval_set['uclassify'].value_counts()
compare = [(actual[i], predicted[i]) for i in predicted.index.values]

In [ ]:
cmp = pd.DataFrame(compare)
cmp.index = predicted.index.values
cmp.columns = ['actual', 'uclassify']
sns.regplot(x=cmp['actual'],y=cmp['uclassify'])

In [ ]:
sns.heatmap(cm, annot=False)

# Conclusion and further research

* Definately an improvement in classification results. 

* Since Jungs cognitive functions are really dichotomies of sensing <-> intuition and thinking <-> feeling, two new classifiers should be created to see if results improve.

* Another important thing is that no pre-processing of the texts have been done. Use TF-IDF to separate out noice from relevant features?



#  Dataset for Kaggle.com

In [67]:
len(blogs)

24331

In [69]:
kaggle = df[["text","func","funcatt"]]
kaggle.columns = ["text","base_function","directed_function"]
kaggle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24331 entries, 1 to 27112
Data columns (total 3 columns):
text                 24331 non-null object
base_function        24331 non-null object
directed_function    24331 non-null object
dtypes: object(3)
memory usage: 760.3+ KB


In [70]:
kaggle.to_csv("../../data/processed/blog_texts_and_cognitive_function.csv", sep=";")

Something is wrong again. The current Kaggle dataset contains 20708 rows. We shouldn't have this many new rows, should we?

In [74]:
df.head(1000)

,text,tokens,func,funcatt,actual_temp,training_set,sntf,sn,tf
2363,Floral High SCM player skins Floral High Maya ...,95,s,se,sf,0,s,s,f
22068,"pv. jemxarstairs cat, eighteen + i wish i coul...",30,s,si,sf,0,s,s,t
22548,Means to a Book End home • Message • Submit • ...,114,s,si,sf,0,s,s,f
15611,scatterbrain. hi my name is panic! at the disc...,132,s,se,sf,0,s,s,f
6105,"HERP DERP Hi, I'm Jacob. My friend, Chelsea, t...",508,s,se,sf,0,s,s,f
11824,The Rose H Show! The Rose H Show! One of these...,252,s,si,st,0,s,s,t
19838,mrs. emma knightley mrs. emma knightley Merile...,282,s,si,sf,0,s,s,f
21341,"MOON TRASH Hello, welcome to my blog! My name ...",444,s,si,sf,0,s,s,f
2793,And What do we say to the god of death? Lexie....,101,s,si,st,0,s,s,f
15646,catch you in the drift home ask submit graphic...,297,s,si,st,0,s,s,f
